In [2]:
import pvlib
from pvlib import pvsystem
import pandas as pd; import numpy as np 
from IPython.display import clear_output
from datetime import datetime, timedelta

In [3]:
import urllib3, socket
from urllib3.connection import HTTPConnection

HTTPConnection.default_socket_options = ( 
     HTTPConnection.default_socket_options + [
     (socket.SOL_SOCKET, socket.SO_SNDBUF, 2000000), #1MB in byte
     (socket.SOL_SOCKET, socket.SO_RCVBUF, 2000000)
])

In [4]:
weather  = pd.read_csv("0_Data/0_Weather/0_weather.csv")
weather = weather[:8760]

comb_data = pd.read_csv("0_Data/Case_1/comb_data.csv")
comb_data = comb_data[::6]



In [31]:
weather

,TDryBul,TDewPoi,relHum,pAtm,HHorIR,HGloHor,HDirNor,HDifHor,winDir,winSpe,nTot,nOpa,ceiHei,time
0,-12.2,-16.1,73.0,99500.0,0.0,0.0,0.0,0.0,270.0,2.6,9.0,9.0,2740.0,0.0
1,-11.7,-15.6,73.0,99600.0,0.0,0.0,0.0,0.0,250.0,2.6,10.0,10.0,1680.0,3600.0
2,-11.1,-15.0,73.0,99500.0,0.0,0.0,0.0,0.0,240.0,2.1,10.0,10.0,1400.0,7200.0
3,-11.1,-15.0,73.0,99500.0,0.0,0.0,0.0,0.0,230.0,2.6,10.0,10.0,1400.0,10800.0
4,-10.6,-14.4,73.0,99500.0,0.0,0.0,0.0,0.0,240.0,2.6,10.0,10.0,1400.0,14400.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8755,-1.1,-2.8,89.0,97800.0,0.0,0.0,0.0,0.0,250.0,10.8,10.0,10.0,270.0,31518000.0
8756,-3.9,-6.7,81.0,98000.0,0.0,0.0,0.0,0.0,250.0,9.3,10.0,10.0,310.0,31521600.0
8757,-4.4,-7.2,81.0,98100.0,0.0,0.0,0.0,0.0,250.0,9.3,10.0,10.0,340.0,31525200.0
8758,-5.0,-8.3,78.0,98200.0,0.0,0.0,0.0,0.0,270.0,6.7,10.0,10.0,340.0,31528800.0


In [5]:
weather["TDryBul"] = list(comb_data["temp_OA"])
weather["HHorIR"] = list(comb_data["hir_sol"])



In [18]:
len(weather[:8760]['time']/3600)

8760

In [6]:
step = 600 
insert_rows = int(3600/step)  # how many rows to insert

In [7]:
def get_pv_data(day_weather_600,i):
     temp_air = day_weather_600.iloc[i].TDryBul
     dni = day_weather_600.iloc[i].HDirNor
     dhi = day_weather_600.iloc[i].HDifHor
     ghi = day_weather_600.iloc[i].HHorIR 
     w_sp = day_weather_600.iloc[i].winSpe
     return temp_air,dni,dhi,ghi,w_sp


In [8]:
day_no = 1

day_weather = weather.iloc[day_no*24:(day_no+1)*24]
day_weather.index = range(0, insert_rows * len(day_weather), insert_rows)
# create new_df with added rows
day_weather_600 = day_weather.reindex(index = range(insert_rows * len(day_weather)))
day_weather_600 = day_weather_600.interpolate()  

day_weather_600


,TDryBul,TDewPoi,relHum,pAtm,HHorIR,HGloHor,HDirNor,HDifHor,winDir,winSpe,nTot,nOpa,ceiHei,time
0,4.233333,-6.1,85.000000,99000.0,308.000000,0.0,0.0,0.0,170.0,2.100000,2.000000,2.000000,77777.0,86400.0
1,4.177778,-6.1,83.833333,99000.0,307.666667,0.0,0.0,0.0,175.0,2.266667,2.333333,2.166667,77777.0,87000.0
2,4.122222,-6.1,82.666667,99000.0,307.333333,0.0,0.0,0.0,180.0,2.433333,2.666667,2.333333,77777.0,87600.0
3,4.066667,-6.1,81.500000,99000.0,307.000000,0.0,0.0,0.0,185.0,2.600000,3.000000,2.500000,77777.0,88200.0
4,4.011111,-6.1,80.333333,99000.0,306.666667,0.0,0.0,0.0,190.0,2.766667,3.333333,2.666667,77777.0,88800.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
139,5.400000,-0.6,96.000000,97800.0,310.000000,0.0,0.0,0.0,320.0,2.600000,10.000000,10.000000,180.0,169200.0
140,5.400000,-0.6,96.000000,97800.0,310.000000,0.0,0.0,0.0,320.0,2.600000,10.000000,10.000000,180.0,169200.0
141,5.400000,-0.6,96.000000,97800.0,310.000000,0.0,0.0,0.0,320.0,2.600000,10.000000,10.000000,180.0,169200.0
142,5.400000,-0.6,96.000000,97800.0,310.000000,0.0,0.0,0.0,320.0,2.600000,10.000000,10.000000,180.0,169200.0


In [9]:
def get_pv_output(temp_air,dni,dhi,ghi,w_sp,no_of_modules,surface_tilt,azimuth,hours,day_no):
     energies = {}
     
     # coordinates = [(42.8, -88.0, 'Chicago', 182, 'Etc/GMT+6')]  

     coordinates = [(32.2, -84.0, 'Tucson', 182, 'Etc/GMT-5')]   
     sandia_modules = pvlib.pvsystem.retrieve_sam('SandiaMod')

     # print ("Sandia modules: {}".format(sandia_modules))
     sapm_inverters = pvlib.pvsystem.retrieve_sam('cecinverter')
     module = sandia_modules['Canadian_Solar_CS5P_220M___2009_']
     inverter = sapm_inverters['ABB__MICRO_0_25_I_OUTD_US_208__208V_']
     temperature_model_parameters = pvlib.temperature.TEMPERATURE_MODEL_PARAMETERS['sapm']['open_rack_glass_glass']
     

     tmys = []
     for location in coordinates:
          latitude, longitude, name, altitude, timezone = location
          weather = pvlib.iotools.get_pvgis_tmy(latitude, longitude,
                                                  map_variables=True)[0]

          # print ("weather")
          # print (weather[0:48])



          
          weather = weather.iloc[0:1] 
          weather.index = weather.index +timedelta(hours=6)+timedelta(hours=day_no*24+hours)
          
          weather.index.name = "utc_time"
          
          tmys.append(weather)

     # print ("Weather Check")
     # print (weather)
     
     system = {'module': module, 'inverter': inverter,'surface_azimuth': azimuth}    

     weather['dni'] = dni
     weather['ghi'] = ghi
     weather['dhi'] = dhi
     weather['wind_speed'] = w_sp
     weather['temp_air'] = temp_air

     for location, weather in zip(coordinates, tmys):
          latitude, longitude, name, altitude, timezone = location
          system['surface_tilt'] = surface_tilt
          solpos = pvlib.solarposition.get_solarposition(
               time=weather.index,
               latitude=latitude,
               longitude=longitude,
               altitude=altitude, 
               temperature=weather["temp_air"],
               pressure=pvlib.atmosphere.alt2pres(altitude),
          )
          dni_extra = pvlib.irradiance.get_extra_radiation(weather.index)
          airmass = pvlib.atmosphere.get_relative_airmass(solpos['apparent_zenith'])
          pressure = pvlib.atmosphere.alt2pres(altitude)
          am_abs = pvlib.atmosphere.get_absolute_airmass(airmass, pressure)
          aoi = pvlib.irradiance.aoi(
               system['surface_tilt'],
               system['surface_azimuth'],
               solpos["apparent_zenith"],
               solpos["azimuth"],
          )
          total_irradiance = pvlib.irradiance.get_total_irradiance(
               system['surface_tilt'],
               system['surface_azimuth'],
               solpos['apparent_zenith'],
               solpos['azimuth'],
               weather['dni'],
               weather['ghi'],
               weather['dhi'],
               dni_extra=dni_extra,
               model='haydavies',
          )
          cell_temperature = pvlib.temperature.sapm_cell(
               total_irradiance['poa_global'],
               weather["temp_air"],
               weather["wind_speed"],
               **temperature_model_parameters,
          )
          effective_irradiance = pvlib.pvsystem.sapm_effective_irradiance(
               total_irradiance['poa_direct'],
               total_irradiance['poa_diffuse'],
               am_abs,
               aoi,
               module,
          )
          dc = pvlib.pvsystem.sapm(effective_irradiance, cell_temperature, module)
          ac = pvlib.inverter.sandia(dc['v_mp'], dc['p_mp'], inverter)

          if ac[0]<0:
               ac[0]=0

          return ac[0]*no_of_modules

In [22]:
day_weather_600[12*6:12*7]

,TDryBul,TDewPoi,relHum,pAtm,HHorIR,HGloHor,HDirNor,HDifHor,winDir,winSpe,nTot,nOpa,ceiHei,time
72,7.400000,-3.900000,67.000000,98600.000000,314.333333,186.000000,3.0,185.000000,0.000000,0.000000,10.0,10.0,1830.000000,129600.0
73,7.527778,-3.433333,71.166667,98583.333333,311.944444,182.666667,2.5,181.833333,26.666667,0.433333,10.0,10.0,1565.000000,130200.0
74,7.655556,-2.966667,75.333333,98566.666667,309.555556,179.333333,2.0,178.666667,53.333333,0.866667,10.0,10.0,1300.000000,130800.0
75,7.783333,-2.500000,79.500000,98550.000000,307.166667,176.000000,1.5,175.500000,80.000000,1.300000,10.0,10.0,1035.000000,131400.0
76,7.911111,-2.033333,83.666667,98533.333333,304.777778,172.666667,1.0,172.333333,106.666667,1.733333,10.0,10.0,770.000000,132000.0
77,8.038889,-1.566667,87.833333,98516.666667,302.388889,169.333333,0.5,169.166667,133.333333,2.166667,10.0,10.0,505.000000,132600.0
78,8.166667,-1.100000,92.000000,98500.000000,300.000000,166.000000,0.0,166.000000,160.000000,2.600000,10.0,10.0,240.000000,133200.0
79,8.255556,-1.100000,92.666667,98483.333333,300.611111,158.666667,0.0,158.666667,153.333333,2.516667,10.0,10.0,351.666667,133800.0
80,8.344444,-1.100000,93.333333,98466.666667,301.222222,151.333333,0.0,151.333333,146.666667,2.433333,10.0,10.0,463.333333,134400.0
81,8.433333,-1.100000,94.000000,98450.000000,301.833333,144.000000,0.0,144.000000,140.000000,2.350000,10.0,10.0,575.000000,135000.0


In [10]:
coordinates = [(33.7, -84.0, 'Atlanta', 1050, 'Etc/GMT+6')]     


train_days = list(range(270,290)) 
train_days = [num for num in train_days if num % 7 != 0]
 
tot_days = [273] #train_days


for day_no in tot_days:
     day_weather = weather.iloc[day_no*24:(day_no+1)*24]
     pv_df =pd.DataFrame(columns=['south_pv','east_pv','west_pv','car_port_pv'],dtype=np.float64)
     day_weather.index = range(0, insert_rows * len(day_weather), insert_rows)
     # create new_df with added rows
     day_weather_600 = day_weather.reindex(index = range(insert_rows * len(day_weather)))
     day_weather_600 = day_weather_600.interpolate()     
     for i in range(6*24):
          temp_air,dni,dhi,ghi,w_sp = get_pv_data(day_weather_600=day_weather_600,i=i)
          print ("day_no: {},i: {}".format(day_no,i))
          print ("temp air: {}; dni: {}; dhi: {}; ghi: {}; w_sp: {}".format(temp_air,dni,dhi,ghi,w_sp))
          if (i<6*5)or(i>18*6):
               south_pv = 0 
               east_pv = 0 
               west_pv = 0 
               car_port_pv = 0 
          else:
               south_pv = get_pv_output(temp_air=temp_air,
                                   dni=dni,
                                   dhi=dhi,
                                   ghi=ghi,
                                   w_sp=w_sp,
                                   no_of_modules=1,
                                   surface_tilt=46,
                                   azimuth=180,
                                   hours=i*step/3600,
                                   day_no=day_no)
               
               east_pv = get_pv_output(temp_air=temp_air,
                                   dni=dni,
                                   dhi=dhi,
                                   ghi=ghi,
                                   w_sp=w_sp,
                                   no_of_modules=1,
                                   surface_tilt=19,
                                   azimuth=90,
                                   hours=i*step/3600,
                                   day_no=day_no)

               west_pv = get_pv_output(temp_air=temp_air,
                                   dni=dni,
                                   dhi=dhi,
                                   ghi=ghi,
                                   w_sp=w_sp,
                                   no_of_modules=1,
                                   surface_tilt=19,
                                   azimuth=270,
                                   hours=i*step/3600,
                                   day_no=day_no
                                   )

               car_port_pv = get_pv_output(temp_air=temp_air,
                         dni=dni,
                         dhi=dhi,
                         ghi=ghi,
                         w_sp=w_sp,
                         no_of_modules=1,
                         surface_tilt=15,
                         azimuth=180,
                         hours=i*step/3600,
                         day_no=day_no)

          
          df1 = pd.DataFrame({'south_pv':[south_pv],'east_pv':[east_pv],'west_pv':[west_pv],'car_port_pv':[car_port_pv]})
          pv_df = pd.concat([pv_df, df1])
          print (df1)

          if i%6==0:                    
               clear_output(wait=True)


     pv_df.to_pickle("0_Data/1_PV/"+str(day_no)+".pkl")

    

day_no: 273,i: 139
temp air: 20.4; dni: 0.0; dhi: 0.0; ghi: 396.6666666666667; w_sp: 2.1
   south_pv  east_pv  west_pv  car_port_pv
0         0        0        0            0
day_no: 273,i: 140
temp air: 20.4; dni: 0.0; dhi: 0.0; ghi: 396.6666666666667; w_sp: 2.1
   south_pv  east_pv  west_pv  car_port_pv
0         0        0        0            0
day_no: 273,i: 141
temp air: 20.4; dni: 0.0; dhi: 0.0; ghi: 396.6666666666667; w_sp: 2.1
   south_pv  east_pv  west_pv  car_port_pv
0         0        0        0            0
day_no: 273,i: 142
temp air: 20.4; dni: 0.0; dhi: 0.0; ghi: 396.6666666666667; w_sp: 2.1
   south_pv  east_pv  west_pv  car_port_pv
0         0        0        0            0
day_no: 273,i: 143
temp air: 20.4; dni: 0.0; dhi: 0.0; ghi: 396.6666666666667; w_sp: 2.1
   south_pv  east_pv  west_pv  car_port_pv
0         0        0        0            0


In [7]:
def get_pv_output(temp_air,dni,dhi,ghi,w_sp,no_of_modules,surface_tilt,azimuth,hours,day_no):
     energies = {}
     
     # coordinates = [(42.8, -88.0, 'Chicago', 182, 'Etc/GMT+6')]  

     coordinates = [(32.2, -88.0, 'Tucson', 182, 'Etc/GMT+7')]   
     sandia_modules = pvlib.pvsystem.retrieve_sam('SandiaMod')

     # print ("Sandia modules: {}".format(sandia_modules))
     sapm_inverters = pvlib.pvsystem.retrieve_sam('cecinverter')
     module = sandia_modules['Canadian_Solar_CS5P_220M___2009_']
     inverter = sapm_inverters['ABB__MICRO_0_25_I_OUTD_US_208__208V_']
     temperature_model_parameters = pvlib.temperature.TEMPERATURE_MODEL_PARAMETERS['sapm']['open_rack_glass_glass']
     

     tmys = []
     for location in coordinates:
          latitude, longitude, name, altitude, timezone = location
          weather = pvlib.iotools.get_pvgis_tmy(latitude, longitude,
                                                  map_variables=True)[0]

          # print ("weather")
          # print (weather[0:48])



          
          weather = weather.iloc[0:1] 
          weather.index = weather.index +timedelta(hours=6)+timedelta(hours=day_no*24+hours)
          
          weather.index.name = "utc_time"
          
          tmys.append(weather)

     # print ("Weather Check")
     # print (weather)
     
     system = {'module': module, 'inverter': inverter,'surface_azimuth': azimuth}    

     weather['dni'] = dni
     weather['ghi'] = ghi
     weather['dhi'] = dhi
     weather['wind_speed'] = w_sp
     weather['temp_air'] = temp_air

     for location, weather in zip(coordinates, tmys):
          latitude, longitude, name, altitude, timezone = location
          system['surface_tilt'] = surface_tilt
          solpos = pvlib.solarposition.get_solarposition(
               time=weather.index,
               latitude=latitude,
               longitude=longitude,
               altitude=altitude, 
               temperature=weather["temp_air"],
               pressure=pvlib.atmosphere.alt2pres(altitude),
          )
          dni_extra = pvlib.irradiance.get_extra_radiation(weather.index)
          airmass = pvlib.atmosphere.get_relative_airmass(solpos['apparent_zenith'])
          pressure = pvlib.atmosphere.alt2pres(altitude)
          am_abs = pvlib.atmosphere.get_absolute_airmass(airmass, pressure)
          aoi = pvlib.irradiance.aoi(
               system['surface_tilt'],
               system['surface_azimuth'],
               solpos["apparent_zenith"],
               solpos["azimuth"],
          )
          total_irradiance = pvlib.irradiance.get_total_irradiance(
               system['surface_tilt'],
               system['surface_azimuth'],
               solpos['apparent_zenith'],
               solpos['azimuth'],
               weather['dni'],
               weather['ghi'],
               weather['dhi'],
               dni_extra=dni_extra,
               model='haydavies',
          )
          cell_temperature = pvlib.temperature.sapm_cell(
               total_irradiance['poa_global'],
               weather["temp_air"],
               weather["wind_speed"],
               **temperature_model_parameters,
          )
          effective_irradiance = pvlib.pvsystem.sapm_effective_irradiance(
               total_irradiance['poa_direct'],
               total_irradiance['poa_diffuse'],
               am_abs,
               aoi,
               module,
          )
          dc = pvlib.pvsystem.sapm(effective_irradiance, cell_temperature, module)
          ac = pvlib.inverter.sandia(dc['v_mp'], dc['p_mp'], inverter)

          if ac[0]<0:
               ac[0]=0

          return ac[0]*no_of_modules

In [136]:
temp_air = 3
dni = 230
dhi = 400
ghi = 100
w_sp = 3 
hours= 17

get_pv_output(temp_air=temp_air,
                              dni=dni,
                              dhi=dhi,
                              ghi=ghi,
                              w_sp=w_sp,
                              no_of_modules=1,
                              surface_tilt=46,
                              azimuth=180,
                              hours=hours,
                              day_no=day_no)



weather


0.0

In [154]:
day_no=1

pv_df=pd.read_pickle("RL_Data/00_General/01_PV/"+str(day_no)+".pkl")

pv_df.reset_index(drop=True,inplace=True)

pv_df


,south_pv,east_pv,west_pv,car_port_pv
0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0
...,...,...,...,...
283,0.0,0.0,0.0,0.0
284,0.0,0.0,0.0,0.0
285,0.0,0.0,0.0,0.0
286,0.0,0.0,0.0,0.0
